In [1]:
import pandas as pd 
import numpy as np 

In [2]:
import pandas as pd
df  = pd.read_csv("D:\personal Projects\classical ML\gr.noida rent prediction (web scrapping)\greater_noida_rents_massive.csv")
df.head()

,BHK,Price,Location,Area,Furnishing,Tenant_Pref,Full_Title,Page_Found
0,2 BHK,"₹17,000 /month",Mahagun Mywoods Phase 1,850 sqft(79 sqm),NaN,Not Specified,"2 BHK Flat for rent in Sector 16C, Greater Noida",1
1,3 BHK,"₹35,000 /month",ACE Divino,"1,565 sqft(145 sqm)",NaN,Not Specified,3 BHK Flat for rent in Noida Extension,1
2,2 BHK,"₹25,000 /month",Ace City,"1,080 sqft(100 sqm)",NaN,Not Specified,2 BHK Flat for rent in Sector 1 Greater Noida ...,1
3,2 BHK,"₹25,000 /month",C block,"1,615 sqft(150 sqm)",Furnished,Not Specified,2 BHK Independent Builder Floor for rent in Si...,1
4,NaN,"₹55,000 /month",Panchsheel Villas,"1,080 sqft(100 sqm)",Furnished,Not Specified,"5 Bedroom House for rent in Sector 16B, Greate...",1


In [3]:
print('Shape of Data:',df.shape)
print('----'*20,'\n Data Types Info:')
print(df.info())

Shape of Data: (2526, 8)
-------------------------------------------------------------------------------- 
 Data Types Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2526 entries, 0 to 2525
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   BHK          2293 non-null   object
 1   Price        2526 non-null   object
 2   Location     2526 non-null   object
 3   Area         2526 non-null   object
 4   Furnishing   735 non-null    object
 5   Tenant_Pref  2526 non-null   object
 6   Full_Title   2526 non-null   object
 7   Page_Found   2526 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 158.0+ KB
None


# Data Cleaning

In [4]:
print(df.isnull().sum())
print('----'*20,'\n Tenent_pref Value Counts:')
print(df['Tenant_Pref'].value_counts())

BHK             233
Price             0
Location          0
Area              0
Furnishing     1791
Tenant_Pref       0
Full_Title        0
Page_Found        0
dtype: int64
-------------------------------------------------------------------------------- 
 Tenent_pref Value Counts:
Tenant_Pref
Not Specified        2471
Family Preferred       39
Bachelors Allowed      10
Family Only             5
Boys Only               1
Name: count, dtype: int64


In [5]:
import re
import pandas as pd

# list of noisy property words that should NOT be treated as a sector/address
property_words = [
    "flat", "flats", "bhk", "bedroom", "bedrooms", "house", "independent",
    "builder floor", "floor", "studio", "apartment", "room", "rooms",
    "for rent", "rent", "sale", "residential", "resale"
]
prop_regex = r"\b(?:" + "|".join(re.escape(w) for w in property_words) + r")\b"

# 1) Primary targeted extraction for known block names / sectors (require number/letter or known word)
pattern_primary = (
    r"(Sector\s*[A-Za-z0-9]+|"
    r"Alpha\s*\w*|Beta\s*\w*|Gamma\s*\w*|Delta\s*\w*|Chi\s*\w*|"
    r"Phi\s*\w*|Omega\s*\w*|Zeta\s*\w*)"
)

df['Sector'] = df['Full_Title'].str.extract(pattern_primary, expand=False)

# 2) Fallback: text after "in " up to comma, parenthesis, or hyphen (most likely contains the address)
fallback = df['Full_Title'].str.extract(r"in\s+([^,()\-\n]+)", expand=False)

# 3) Fill primary NaNs with fallback
df['Sector'] = df['Sector'].fillna(fallback)

# 4) Remove property-type noise from the extracted value
df['Sector'] = (
    df['Sector']
      .astype(str)                                      # safe cast
      .str.replace(prop_regex, "", regex=True, flags=re.IGNORECASE)  # remove property words
      .str.replace(r"\b(Greater Noida|Noida|Noida Extension|Greater Noida West)\b", "", regex=True, case=False)
      .str.replace(r"[,:\-\(\)]", "", regex=True)     # remove punctuation
      .str.strip()
      .replace({"": pd.NA, "nan": pd.NA})             # empty -> NaN
)

# 5) Validation: keep only if it looks like a block/sector/address.
#    We require either:
#      - the word "Sector" OR
#      - a known block name (Alpha/Beta/Chi/...) OR
#      - at least one digit in the result (e.g. "16C", "4", "Block 12")
valid_pattern = re.compile(r"(?i)^(?:.*Sector.*|.*Alpha.*|.*Beta.*|.*Gamma.*|.*Chi.*|.*Delta.*|.*Omega.*|.*Zeta.*|.*\d.*)$")

mask_valid = df['Sector'].notna() & df['Sector'].str.match(valid_pattern)
df.loc[~mask_valid, 'Sector'] = pd.NA

# 6) Optional: tidy casing and normalize "sector" words
df['Sector'] = df['Sector'].str.replace(r"^sector\s+", "Sector ", flags=re.IGNORECASE, regex=True)
df['Sector'] = df['Sector'].str.title()

# Quick checks
print("Remaining NaNs in Sector:", df['Sector'].isnull().sum())
print("Sample extracted values (non-null):")
print(df['Sector'].dropna().unique()[:50])


Remaining NaNs in Sector: 341
Sample extracted values (non-null):
['Sector 16C' 'Sector 1' 'Sigma 4' 'Sector 16B' 'Sector 10'
 'Techzone 4  West' 'Sector 27' 'Sector Zeta' 'Sector 16' 'Sector Chi'
 'Sector 4' 'Sector' 'Sector Pi' 'Sector 2' 'Sector Xu' 'Sector P4'
 'Sector 31' 'Sector Omicron' 'Techzone 4' 'Eta 2' 'Nh 91' 'Sector Mu'
 'Delta 1' 'Sector 3' 'Alpha 2' 'Sector Phi' 'Sector 22D' 'Gamma 2'
 'Beta 1' 'Ecotech 3' 'Knowledge Park 3' 'Delta I' 'Sector P3' 'Sigma 2'
 'Site 5' 'Beta 2' 'Rho 1 Sector' 'Sector 25' 'Ecotech 1 Extension'
 'Chi Phi' 'Sigma 1' 'Delta 3' 'Sector Omega' 'Alpha 1' 'Gamma 1']


In [6]:
df.isnull().sum()

BHK             233
Price             0
Location          0
Area              0
Furnishing     1791
Tenant_Pref       0
Full_Title        0
Page_Found        0
Sector          341
dtype: int64

In [7]:
df.dropna(subset=['Sector'], inplace=True)
df.dropna(subset=['BHK'], inplace=True)
df.isnull().sum()

BHK               0
Price             0
Location          0
Area              0
Furnishing     1529
Tenant_Pref       0
Full_Title        0
Page_Found        0
Sector            0
dtype: int64

In [8]:
df.head()


,BHK,Price,Location,Area,Furnishing,Tenant_Pref,Full_Title,Page_Found,Sector
0,2 BHK,"₹17,000 /month",Mahagun Mywoods Phase 1,850 sqft(79 sqm),NaN,Not Specified,"2 BHK Flat for rent in Sector 16C, Greater Noida",1,Sector 16C
2,2 BHK,"₹25,000 /month",Ace City,"1,080 sqft(100 sqm)",NaN,Not Specified,2 BHK Flat for rent in Sector 1 Greater Noida ...,1,Sector 1
3,2 BHK,"₹25,000 /month",C block,"1,615 sqft(150 sqm)",Furnished,Not Specified,2 BHK Independent Builder Floor for rent in Si...,1,Sigma 4
5,3 BHK,"₹21,000 /month",Arihant Abode,"1,160 sqft(108 sqm)",NaN,Not Specified,3 BHK Flat for rent in Sector 10 Greater Noida...,1,Sector 10
6,4 BHK,"₹26,000 /month",La Residentia,"1,538 sqft(143 sqm)",Semi-Furnished,Not Specified,4 BHK Flat for rent in Techzone 4 Greater Noid...,1,Techzone 4 West


In [9]:
cols_to_drop = [
    'Location',
    'Furnishing',
    'Tenant_Pref',
    'Full_Title',
    'Page_Found'
]

df = df.drop(columns=cols_to_drop, errors='ignore')
df.head()

,BHK,Price,Area,Sector
0,2 BHK,"₹17,000 /month",850 sqft(79 sqm),Sector 16C
2,2 BHK,"₹25,000 /month","1,080 sqft(100 sqm)",Sector 1
3,2 BHK,"₹25,000 /month","1,615 sqft(150 sqm)",Sigma 4
5,3 BHK,"₹21,000 /month","1,160 sqft(108 sqm)",Sector 10
6,4 BHK,"₹26,000 /month","1,538 sqft(143 sqm)",Techzone 4 West


In [10]:
# Changing BHK col into int
df['BHK'] = df['BHK'].str.extract(r'(\d+)').astype(int)

#changing Price Col into Int
df['Price (Rupee)'] = (
    df['Price']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.extract(r'(\d+)')[0]
    .astype(int)
)
# changing Area col into int
df['Area_sqft'] = df['Area'].str.extract(r'(\d+)\s*sqft')[0].astype(int)

#dropping original cols
cols_to_drop2 = [
    'Price',
    'Area'   
]
# dropping original cols
df = df.drop(columns=cols_to_drop2, errors='ignore')
df.head()


,BHK,Sector,Price (Rupee),Area_sqft
0,2,Sector 16C,17000,850
2,2,Sector 1,25000,80
3,2,Sigma 4,25000,615
5,3,Sector 10,21000,160
6,4,Techzone 4 West,26000,538


In [11]:
print(df.info())
print('---'*20,)
print('\n count of catagory in Sector Column is' , len(df['Sector'].unique()))

<class 'pandas.core.frame.DataFrame'>
Index: 2007 entries, 0 to 2525
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BHK            2007 non-null   int32 
 1   Sector         2007 non-null   object
 2   Price (Rupee)  2007 non-null   int32 
 3   Area_sqft      2007 non-null   int32 
dtypes: int32(3), object(1)
memory usage: 54.9+ KB
None
------------------------------------------------------------

 count of catagory in Sector Column is 32


In [12]:
df['Sector'].unique()

array(['Sector 16C', 'Sector 1', 'Sigma 4', 'Sector 10',
       'Techzone 4  West', 'Sector 27', 'Sector 16', 'Sector Zeta',
       'Sector 16B', 'Sector Chi', 'Sector 4', 'Sector', 'Sector Pi',
       'Sector 2', 'Sector Xu', 'Sector P4', 'Sector 31',
       'Sector Omicron', 'Techzone 4', 'Eta 2', 'Sector Mu', 'Sector 3',
       'Alpha 2', 'Sector Phi', 'Ecotech 3', 'Gamma 2', 'Sector 22D',
       'Sigma 2', 'Beta 1', 'Beta 2', 'Rho 1 Sector', 'Sector 25'],
      dtype=object)

Feature Engreening

In [18]:
# Creating a new feature Area per BHK
df['Area_Per_BHK'] = df['Area_sqft'] / df['BHK']

# Log Transformation of Price for better distribution
df['log_Price'] = np.log1p(df['Price (Rupee)'])

df.head()

,BHK,Sector,Price (Rupee),Area_sqft,Area_Per_BHK,log_Price
0,2,Sector 16C,17000,850,425.000000,9.741027
2,2,Sector 1,25000,80,40.000000,10.126671
3,2,Sigma 4,25000,615,307.500000,10.126671
5,3,Sector 10,21000,160,53.333333,9.952325
6,4,Techzone 4 West,26000,538,134.500000,10.165890


In [19]:
# Splliting the data into X and y   (80% Train, 20% Test)
X = df.drop(columns=['Price (Rupee)', 'log_Price'])

y = df['log_Price'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Train Shape: {X_train.shape}, Test Shape: {X_test.shape}")

Train Shape: (1605, 4), Test Shape: (402, 4)


In [20]:
# We calculate the mean target value for each sector using ONLY Training data
# to prevent data leakage.

# Temporary dataframe to calculate means
train_temp = X_train.copy()
train_temp['target'] = y_train

# Calculate mean log_Price for each Sector
sector_means = train_temp.groupby('Sector')['target'].mean()

# Map these means to the Training set
X_train['Sector_Encoded'] = X_train['Sector'].map(sector_means)

# Map these SAME means to the Testing set
X_test['Sector_Encoded'] = X_test['Sector'].map(sector_means)

# Handle potential NaNs in Test set (Sectors in Test that weren't in Train)
# Fill with the global average of the training target
global_mean = y_train.mean()
X_test['Sector_Encoded'] = X_test['Sector_Encoded'].fillna(global_mean)

# Optional: Drop the original text 'Sector' column now that we have the numbers
X_train = X_train.drop(columns=['Sector'])
X_test = X_test.drop(columns=['Sector'])

# Display the first few rows to verify
print("\nProcessed Training Data:")
print(X_train.head())


Processed Training Data:
      BHK  Area_sqft  Area_Per_BHK  Sector_Encoded
1145    3        185     61.666667       10.083363
326     4        950    237.500000       10.503875
717     2        975    487.500000        9.989036
1676    3        300    100.000000        9.981102
304     2        920    460.000000       10.050724
